In [4]:
import pandas as pd
import string
import re
import nltk
from nltk.tokenize import regexp_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer, SnowballStemmer, LancasterStemmer
import spacy

# Read Data

In [5]:
df = pd.read_csv('/content/news_articles_txt100.csv', index_col=0)
print(df.shape)
df.head()

(100, 6)


,headline,url,source,source_url,category,text
0,Manappuram Finance to consider fund raising op...,https://www.business-standard.com/article/news...,Business Standard,https://www.business-standard.com,finance,Manappuram Finance said the company may consid...
1,PayU aims to provide full-stack financial serv...,https://www.business-standard.com/article/comp...,Business Standard,https://www.business-standard.com,finance,"Finance, a digital lending player backed by Pr..."
2,L&T Finance in talks with HSBC to sell its mut...,https://www.businesstoday.in/latest/corporate/...,Business Today,https://www.businesstoday.in,finance,L&T Finance Holdings is in active talks with f...
3,Tally Solutions seeks small finance banking li...,https://economictimes.indiatimes.com/industry/...,Economic Times,https://economictimes.indiatimes.com,finance,Financial technology company Tally Solutions a...
4,China Finance Online announces changes to the ...,https://ibsintelligence.com/ibsi-news/china-fi...,IBS Intelligence,https://ibsintelligence.com,finance,"China Finance Online Co. Limited, a web-based ..."


In [6]:
df.text[0]

'Manappuram Finance said the company may consider and approve issuances of Debt Securities during September 2021.\n\nThe company is considering various options for raising funds through borrowings including by the way of issuance of various debt securities in onshore / offshore securities market by Public Issue, on Private Placement Basis or through issuing Commercial Papers.\n\nBased on the prevailing market conditions, the Board of Directors / Financial Resources and Management Committee / Debenture Committee of the Board of Directors of the Company may consider and approve issuances of Debt Securities during the month of September, 2021, subject to such terms and conditions including the issue price of debt securities, as the Board / respective Committee may deem fit.\n\nShares of Manappuram Finance fell 0.74% to settle at Rs 160.25 on Friday, 27 August 2021.\n\nManappuram Finance is a gold loans offering non-banking financial company, engaged in providing finance against used house

In [7]:
def get_size(field):
    if type(df[field]) == str:
        siz = df[field].str.len().sum()
    else:
        siz = sum(df[field].apply(lambda x: len(''.join(x))))
    print(f"The size of the corpus = {siz} characters")

def get_terms(field):
    terms = set()
    for lis in df[field]:
        for term in lis:
            terms.add(term)
    print(f"The number of terms = {len(terms)}")

In [8]:
get_size('text')

The size of the corpus = 347516 characters


# Preprocessing

## Normalization

### Punctuation Removal

In [9]:
def remove_punctuation(text):
    text = text.replace('\n',' ')
    punctuationfree = "".join([i for i in text if i not in string.punctuation])
    return punctuationfree

df['clean_text'] = df['text'].apply(lambda x:remove_punctuation(x))
df.head()

,headline,url,source,source_url,category,text,clean_text
0,Manappuram Finance to consider fund raising op...,https://www.business-standard.com/article/news...,Business Standard,https://www.business-standard.com,finance,Manappuram Finance said the company may consid...,Manappuram Finance said the company may consid...
1,PayU aims to provide full-stack financial serv...,https://www.business-standard.com/article/comp...,Business Standard,https://www.business-standard.com,finance,"Finance, a digital lending player backed by Pr...",Finance a digital lending player backed by Pro...
2,L&T Finance in talks with HSBC to sell its mut...,https://www.businesstoday.in/latest/corporate/...,Business Today,https://www.businesstoday.in,finance,L&T Finance Holdings is in active talks with f...,LT Finance Holdings is in active talks with fi...
3,Tally Solutions seeks small finance banking li...,https://economictimes.indiatimes.com/industry/...,Economic Times,https://economictimes.indiatimes.com,finance,Financial technology company Tally Solutions a...,Financial technology company Tally Solutions a...
4,China Finance Online announces changes to the ...,https://ibsintelligence.com/ibsi-news/china-fi...,IBS Intelligence,https://ibsintelligence.com,finance,"China Finance Online Co. Limited, a web-based ...",China Finance Online Co Limited a webbased fin...


In [10]:
df.clean_text[0]

'Manappuram Finance said the company may consider and approve issuances of Debt Securities during September 2021  The company is considering various options for raising funds through borrowings including by the way of issuance of various debt securities in onshore  offshore securities market by Public Issue on Private Placement Basis or through issuing Commercial Papers  Based on the prevailing market conditions the Board of Directors  Financial Resources and Management Committee  Debenture Committee of the Board of Directors of the Company may consider and approve issuances of Debt Securities during the month of September 2021 subject to such terms and conditions including the issue price of debt securities as the Board  respective Committee may deem fit  Shares of Manappuram Finance fell 074 to settle at Rs 16025 on Friday 27 August 2021  Manappuram Finance is a gold loans offering nonbanking financial company engaged in providing finance against used household gold ornaments  Powere

In [11]:
get_size('clean_text')

The size of the corpus = 336211 characters


### Case folding

In [12]:
df['lc_text'] = df['clean_text'].apply(lambda x: x.lower())
df.lc_text[0]

'manappuram finance said the company may consider and approve issuances of debt securities during september 2021  the company is considering various options for raising funds through borrowings including by the way of issuance of various debt securities in onshore  offshore securities market by public issue on private placement basis or through issuing commercial papers  based on the prevailing market conditions the board of directors  financial resources and management committee  debenture committee of the board of directors of the company may consider and approve issuances of debt securities during the month of september 2021 subject to such terms and conditions including the issue price of debt securities as the board  respective committee may deem fit  shares of manappuram finance fell 074 to settle at rs 16025 on friday 27 august 2021  manappuram finance is a gold loans offering nonbanking financial company engaged in providing finance against used household gold ornaments  powere

In [13]:
get_size('lc_text')

The size of the corpus = 336211 characters


## Tokenization

In [14]:
def tokenization(text):
    tokens = regexp_tokenize(text, "[\w']+")
    return tokens

df['text_token'] = df['text'].apply(lambda x: tokenization(x))
df['clean_text_token'] = df['clean_text'].apply(lambda x: tokenization(x))
df['lc_text_token'] = df['lc_text'].apply(lambda x: tokenization(x))
df.head()

,headline,url,source,source_url,category,text,clean_text,lc_text,text_token,clean_text_token,lc_text_token
0,Manappuram Finance to consider fund raising op...,https://www.business-standard.com/article/news...,Business Standard,https://www.business-standard.com,finance,Manappuram Finance said the company may consid...,Manappuram Finance said the company may consid...,manappuram finance said the company may consid...,"[Manappuram, Finance, said, the, company, may,...","[Manappuram, Finance, said, the, company, may,...","[manappuram, finance, said, the, company, may,..."
1,PayU aims to provide full-stack financial serv...,https://www.business-standard.com/article/comp...,Business Standard,https://www.business-standard.com,finance,"Finance, a digital lending player backed by Pr...",Finance a digital lending player backed by Pro...,finance a digital lending player backed by pro...,"[Finance, a, digital, lending, player, backed,...","[Finance, a, digital, lending, player, backed,...","[finance, a, digital, lending, player, backed,..."
2,L&T Finance in talks with HSBC to sell its mut...,https://www.businesstoday.in/latest/corporate/...,Business Today,https://www.businesstoday.in,finance,L&T Finance Holdings is in active talks with f...,LT Finance Holdings is in active talks with fi...,lt finance holdings is in active talks with fi...,"[L, T, Finance, Holdings, is, in, active, talk...","[LT, Finance, Holdings, is, in, active, talks,...","[lt, finance, holdings, is, in, active, talks,..."
3,Tally Solutions seeks small finance banking li...,https://economictimes.indiatimes.com/industry/...,Economic Times,https://economictimes.indiatimes.com,finance,Financial technology company Tally Solutions a...,Financial technology company Tally Solutions a...,financial technology company tally solutions a...,"[Financial, technology, company, Tally, Soluti...","[Financial, technology, company, Tally, Soluti...","[financial, technology, company, tally, soluti..."
4,China Finance Online announces changes to the ...,https://ibsintelligence.com/ibsi-news/china-fi...,IBS Intelligence,https://ibsintelligence.com,finance,"China Finance Online Co. Limited, a web-based ...",China Finance Online Co Limited a webbased fin...,china finance online co limited a webbased fin...,"[China, Finance, Online, Co, Limited, a, web, ...","[China, Finance, Online, Co, Limited, a, webba...","[china, finance, online, co, limited, a, webba..."


In [15]:
print(df.text_token[0])

['Manappuram', 'Finance', 'said', 'the', 'company', 'may', 'consider', 'and', 'approve', 'issuances', 'of', 'Debt', 'Securities', 'during', 'September', '2021', 'The', 'company', 'is', 'considering', 'various', 'options', 'for', 'raising', 'funds', 'through', 'borrowings', 'including', 'by', 'the', 'way', 'of', 'issuance', 'of', 'various', 'debt', 'securities', 'in', 'onshore', 'offshore', 'securities', 'market', 'by', 'Public', 'Issue', 'on', 'Private', 'Placement', 'Basis', 'or', 'through', 'issuing', 'Commercial', 'Papers', 'Based', 'on', 'the', 'prevailing', 'market', 'conditions', 'the', 'Board', 'of', 'Directors', 'Financial', 'Resources', 'and', 'Management', 'Committee', 'Debenture', 'Committee', 'of', 'the', 'Board', 'of', 'Directors', 'of', 'the', 'Company', 'may', 'consider', 'and', 'approve', 'issuances', 'of', 'Debt', 'Securities', 'during', 'the', 'month', 'of', 'September', '2021', 'subject', 'to', 'such', 'terms', 'and', 'conditions', 'including', 'the', 'issue', 'price

In [16]:
print('Initially - ')
get_terms('text_token')
get_size('text')
print()
print('After removing punctuations - ')
get_terms('clean_text_token')
get_size('clean_text')
print()
print('After case folding - ')
get_terms('lc_text_token')
get_size('lc_text')
print()

Initially - 
The number of terms = 10238
The size of the corpus = 347516 characters

After removing punctuations - 
The number of terms = 10530
The size of the corpus = 336211 characters

After case folding - 
The number of terms = 9431
The size of the corpus = 336211 characters



## Stopwords removal

In [17]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
stopwords[0:10]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

In [18]:
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

df['no_sw_text']= df['lc_text_token'].apply(lambda x:remove_stopwords(x))
df.head()

,headline,url,source,source_url,category,text,clean_text,lc_text,text_token,clean_text_token,lc_text_token,no_sw_text
0,Manappuram Finance to consider fund raising op...,https://www.business-standard.com/article/news...,Business Standard,https://www.business-standard.com,finance,Manappuram Finance said the company may consid...,Manappuram Finance said the company may consid...,manappuram finance said the company may consid...,"[Manappuram, Finance, said, the, company, may,...","[Manappuram, Finance, said, the, company, may,...","[manappuram, finance, said, the, company, may,...","[manappuram, finance, said, company, may, cons..."
1,PayU aims to provide full-stack financial serv...,https://www.business-standard.com/article/comp...,Business Standard,https://www.business-standard.com,finance,"Finance, a digital lending player backed by Pr...",Finance a digital lending player backed by Pro...,finance a digital lending player backed by pro...,"[Finance, a, digital, lending, player, backed,...","[Finance, a, digital, lending, player, backed,...","[finance, a, digital, lending, player, backed,...","[finance, digital, lending, player, backed, pr..."
2,L&T Finance in talks with HSBC to sell its mut...,https://www.businesstoday.in/latest/corporate/...,Business Today,https://www.businesstoday.in,finance,L&T Finance Holdings is in active talks with f...,LT Finance Holdings is in active talks with fi...,lt finance holdings is in active talks with fi...,"[L, T, Finance, Holdings, is, in, active, talk...","[LT, Finance, Holdings, is, in, active, talks,...","[lt, finance, holdings, is, in, active, talks,...","[lt, finance, holdings, active, talks, financi..."
3,Tally Solutions seeks small finance banking li...,https://economictimes.indiatimes.com/industry/...,Economic Times,https://economictimes.indiatimes.com,finance,Financial technology company Tally Solutions a...,Financial technology company Tally Solutions a...,financial technology company tally solutions a...,"[Financial, technology, company, Tally, Soluti...","[Financial, technology, company, Tally, Soluti...","[financial, technology, company, tally, soluti...","[financial, technology, company, tally, soluti..."
4,China Finance Online announces changes to the ...,https://ibsintelligence.com/ibsi-news/china-fi...,IBS Intelligence,https://ibsintelligence.com,finance,"China Finance Online Co. Limited, a web-based ...",China Finance Online Co Limited a webbased fin...,china finance online co limited a webbased fin...,"[China, Finance, Online, Co, Limited, a, web, ...","[China, Finance, Online, Co, Limited, a, webba...","[china, finance, online, co, limited, a, webba...","[china, finance, online, co, limited, webbased..."


In [19]:
print(df.no_sw_text[0])

['manappuram', 'finance', 'said', 'company', 'may', 'consider', 'approve', 'issuances', 'debt', 'securities', 'september', '2021', 'company', 'considering', 'various', 'options', 'raising', 'funds', 'borrowings', 'including', 'way', 'issuance', 'various', 'debt', 'securities', 'onshore', 'offshore', 'securities', 'market', 'public', 'issue', 'private', 'placement', 'basis', 'issuing', 'commercial', 'papers', 'based', 'prevailing', 'market', 'conditions', 'board', 'directors', 'financial', 'resources', 'management', 'committee', 'debenture', 'committee', 'board', 'directors', 'company', 'may', 'consider', 'approve', 'issuances', 'debt', 'securities', 'month', 'september', '2021', 'subject', 'terms', 'conditions', 'including', 'issue', 'price', 'debt', 'securities', 'board', 'respective', 'committee', 'may', 'deem', 'fit', 'shares', 'manappuram', 'finance', 'fell', '074', 'settle', 'rs', '16025', 'friday', '27', 'august', '2021', 'manappuram', 'finance', 'gold', 'loans', 'offering', 'non

In [20]:
get_size('no_sw_text')

The size of the corpus = 219075 characters


## Stemming

In [21]:
porter_stemmer = PorterStemmer()
snowball_stemmer = SnowballStemmer(language='english')
lancaster_stemmer = LancasterStemmer()

def stemming(text, stemmer):
    stem_text = [stemmer.stem(word) for word in text]
    return stem_text

df['text_stem'] = df['no_sw_text'].apply(lambda x: stemming(x, porter_stemmer))
df.head()

,headline,url,source,source_url,category,text,clean_text,lc_text,text_token,clean_text_token,lc_text_token,no_sw_text,text_stem
0,Manappuram Finance to consider fund raising op...,https://www.business-standard.com/article/news...,Business Standard,https://www.business-standard.com,finance,Manappuram Finance said the company may consid...,Manappuram Finance said the company may consid...,manappuram finance said the company may consid...,"[Manappuram, Finance, said, the, company, may,...","[Manappuram, Finance, said, the, company, may,...","[manappuram, finance, said, the, company, may,...","[manappuram, finance, said, company, may, cons...","[manappuram, financ, said, compani, may, consi..."
1,PayU aims to provide full-stack financial serv...,https://www.business-standard.com/article/comp...,Business Standard,https://www.business-standard.com,finance,"Finance, a digital lending player backed by Pr...",Finance a digital lending player backed by Pro...,finance a digital lending player backed by pro...,"[Finance, a, digital, lending, player, backed,...","[Finance, a, digital, lending, player, backed,...","[finance, a, digital, lending, player, backed,...","[finance, digital, lending, player, backed, pr...","[financ, digit, lend, player, back, prosu, ven..."
2,L&T Finance in talks with HSBC to sell its mut...,https://www.businesstoday.in/latest/corporate/...,Business Today,https://www.businesstoday.in,finance,L&T Finance Holdings is in active talks with f...,LT Finance Holdings is in active talks with fi...,lt finance holdings is in active talks with fi...,"[L, T, Finance, Holdings, is, in, active, talk...","[LT, Finance, Holdings, is, in, active, talks,...","[lt, finance, holdings, is, in, active, talks,...","[lt, finance, holdings, active, talks, financi...","[lt, financ, hold, activ, talk, financi, servi..."
3,Tally Solutions seeks small finance banking li...,https://economictimes.indiatimes.com/industry/...,Economic Times,https://economictimes.indiatimes.com,finance,Financial technology company Tally Solutions a...,Financial technology company Tally Solutions a...,financial technology company tally solutions a...,"[Financial, technology, company, Tally, Soluti...","[Financial, technology, company, Tally, Soluti...","[financial, technology, company, tally, soluti...","[financial, technology, company, tally, soluti...","[financi, technolog, compani, talli, solut, co..."
4,China Finance Online announces changes to the ...,https://ibsintelligence.com/ibsi-news/china-fi...,IBS Intelligence,https://ibsintelligence.com,finance,"China Finance Online Co. Limited, a web-based ...",China Finance Online Co Limited a webbased fin...,china finance online co limited a webbased fin...,"[China, Finance, Online, Co, Limited, a, web, ...","[China, Finance, Online, Co, Limited, a, webba...","[china, finance, online, co, limited, a, webba...","[china, finance, online, co, limited, webbased...","[china, financ, onlin, co, limit, webbas, fina..."


In [22]:
print(df.text_stem[0])

['manappuram', 'financ', 'said', 'compani', 'may', 'consid', 'approv', 'issuanc', 'debt', 'secur', 'septemb', '2021', 'compani', 'consid', 'variou', 'option', 'rais', 'fund', 'borrow', 'includ', 'way', 'issuanc', 'variou', 'debt', 'secur', 'onshor', 'offshor', 'secur', 'market', 'public', 'issu', 'privat', 'placement', 'basi', 'issu', 'commerci', 'paper', 'base', 'prevail', 'market', 'condit', 'board', 'director', 'financi', 'resourc', 'manag', 'committe', 'debentur', 'committe', 'board', 'director', 'compani', 'may', 'consid', 'approv', 'issuanc', 'debt', 'secur', 'month', 'septemb', '2021', 'subject', 'term', 'condit', 'includ', 'issu', 'price', 'debt', 'secur', 'board', 'respect', 'committe', 'may', 'deem', 'fit', 'share', 'manappuram', 'financ', 'fell', '074', 'settl', 'rs', '16025', 'friday', '27', 'august', '2021', 'manappuram', 'financ', 'gold', 'loan', 'offer', 'nonbank', 'financi', 'compani', 'engag', 'provid', 'financ', 'use', 'household', 'gold', 'ornament', 'power', 'capit'

In [23]:
get_terms('text_stem')
get_size('text_stem')

The number of terms = 7097
The size of the corpus = 189381 characters


## Lemmatization

In [24]:
nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()
spacy_lemm = spacy.load('en_core_web_sm')

def lemmatizer(text):
    lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
    return lemm_text

def spacy_lemmatizer(text):
    lemm_text = [spacy_lemm(word)[0].lemma_ for word in text]
    return lemm_text

df['text_lemm'] = df['no_sw_text'].apply(lambda x:lemmatizer(x))
df.head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


,headline,url,source,source_url,category,text,clean_text,lc_text,text_token,clean_text_token,lc_text_token,no_sw_text,text_stem,text_lemm
0,Manappuram Finance to consider fund raising op...,https://www.business-standard.com/article/news...,Business Standard,https://www.business-standard.com,finance,Manappuram Finance said the company may consid...,Manappuram Finance said the company may consid...,manappuram finance said the company may consid...,"[Manappuram, Finance, said, the, company, may,...","[Manappuram, Finance, said, the, company, may,...","[manappuram, finance, said, the, company, may,...","[manappuram, finance, said, company, may, cons...","[manappuram, financ, said, compani, may, consi...","[manappuram, finance, said, company, may, cons..."
1,PayU aims to provide full-stack financial serv...,https://www.business-standard.com/article/comp...,Business Standard,https://www.business-standard.com,finance,"Finance, a digital lending player backed by Pr...",Finance a digital lending player backed by Pro...,finance a digital lending player backed by pro...,"[Finance, a, digital, lending, player, backed,...","[Finance, a, digital, lending, player, backed,...","[finance, a, digital, lending, player, backed,...","[finance, digital, lending, player, backed, pr...","[financ, digit, lend, player, back, prosu, ven...","[finance, digital, lending, player, backed, pr..."
2,L&T Finance in talks with HSBC to sell its mut...,https://www.businesstoday.in/latest/corporate/...,Business Today,https://www.businesstoday.in,finance,L&T Finance Holdings is in active talks with f...,LT Finance Holdings is in active talks with fi...,lt finance holdings is in active talks with fi...,"[L, T, Finance, Holdings, is, in, active, talk...","[LT, Finance, Holdings, is, in, active, talks,...","[lt, finance, holdings, is, in, active, talks,...","[lt, finance, holdings, active, talks, financi...","[lt, financ, hold, activ, talk, financi, servi...","[lt, finance, holding, active, talk, financial..."
3,Tally Solutions seeks small finance banking li...,https://economictimes.indiatimes.com/industry/...,Economic Times,https://economictimes.indiatimes.com,finance,Financial technology company Tally Solutions a...,Financial technology company Tally Solutions a...,financial technology company tally solutions a...,"[Financial, technology, company, Tally, Soluti...","[Financial, technology, company, Tally, Soluti...","[financial, technology, company, tally, soluti...","[financial, technology, company, tally, soluti...","[financi, technolog, compani, talli, solut, co...","[financial, technology, company, tally, soluti..."
4,China Finance Online announces changes to the ...,https://ibsintelligence.com/ibsi-news/china-fi...,IBS Intelligence,https://ibsintelligence.com,finance,"China Finance Online Co. Limited, a web-based ...",China Finance Online Co Limited a webbased fin...,china finance online co limited a webbased fin...,"[China, Finance, Online, Co, Limited, a, web, ...","[China, Finance, Online, Co, Limited, a, webba...","[china, finance, online, co, limited, a, webba...","[china, finance, online, co, limited, webbased...","[china, financ, onlin, co, limit, webbas, fina...","[china, finance, online, co, limited, webbased..."


In [25]:
print(df.text_lemm[0])

['manappuram', 'finance', 'said', 'company', 'may', 'consider', 'approve', 'issuance', 'debt', 'security', 'september', '2021', 'company', 'considering', 'various', 'option', 'raising', 'fund', 'borrowing', 'including', 'way', 'issuance', 'various', 'debt', 'security', 'onshore', 'offshore', 'security', 'market', 'public', 'issue', 'private', 'placement', 'basis', 'issuing', 'commercial', 'paper', 'based', 'prevailing', 'market', 'condition', 'board', 'director', 'financial', 'resource', 'management', 'committee', 'debenture', 'committee', 'board', 'director', 'company', 'may', 'consider', 'approve', 'issuance', 'debt', 'security', 'month', 'september', '2021', 'subject', 'term', 'condition', 'including', 'issue', 'price', 'debt', 'security', 'board', 'respective', 'committee', 'may', 'deem', 'fit', 'share', 'manappuram', 'finance', 'fell', '074', 'settle', 'r', '16025', 'friday', '27', 'august', '2021', 'manappuram', 'finance', 'gold', 'loan', 'offering', 'nonbanking', 'financial', 'c

In [26]:
get_size('text_lemm')
get_terms('text_lemm')

The size of the corpus = 214699 characters
The number of terms = 8493


In [27]:
print('Initially - ')
get_terms('text_token')
get_size('text')
print()
print('After removing punctuations - ')
get_terms('clean_text_token')
get_size('clean_text')
print()
print('After case folding - ')
get_terms('lc_text_token')
get_size('lc_text')
print()
print('After tokenization - ')
get_terms('lc_text_token')
get_size('lc_text_token')
print()
print('After stopwords removal - ')
get_terms('no_sw_text')
get_size('no_sw_text')
print()
print('After stemming - ')
get_terms('text_stem')
get_size('text_stem')
print()
print('After lemmatization - ')
get_terms('text_lemm')
get_size('text_lemm')
print()

Initially - 
The number of terms = 10238
The size of the corpus = 347516 characters

After removing punctuations - 
The number of terms = 10530
The size of the corpus = 336211 characters

After case folding - 
The number of terms = 9431
The size of the corpus = 336211 characters

After tokenization - 
The number of terms = 9431
The size of the corpus = 277687 characters

After stopwords removal - 
The number of terms = 9296
The size of the corpus = 219075 characters

After stemming - 
The number of terms = 7097
The size of the corpus = 189381 characters

After lemmatization - 
The number of terms = 8493
The size of the corpus = 214699 characters



# Inverse Index construction

## List

In [28]:
class Inv_index_list:
    def __init__(self, field):
        self.field = field
        self.index = []
        self.docs = []
        self.construct_index()

    def construct_index(self):
        for i, doc in enumerate(df[self.field]):
            for term in doc:
                if term not in self.index:
                    self.index.append(term)
                    self.docs.append([])
                self.docs[self.index.index(term)].append(i) 
                    
    def get(self, q):
        if q not in self.index:
            return []
        return self.docs[self.index.index(q)]
    
    def delete(self, q):
        if q in self.index:
            del self.docs[self.index.index(q)]
            self.index.remove(q)

## Hashmap

In [29]:
class Inv_index_dict:
    def __init__(self, field):
        self.field = field
        self.index = {}
        self.construct_index()

    def construct_index(self):
        for i, doc in enumerate(df[self.field]):
            for term in doc:
                if term not in self.index:
                    self.index[term] = [i]
                elif self.index[term][-1] != i:
                    self.index[term].append(i)
                    
    def get(self, q):
        return self.index.get(q, [])

    def delete(self, q):
        if q in self.index:
            del self.index[q]

## Trees

In [30]:
from sortedcontainers import SortedDict
class Inv_index_tree:
    def __init__(self, field):
        self.field = field
        self.index = SortedDict()
        self.construct_index()

    def construct_index(self):
        for i, doc in enumerate(df[self.field]):
            for term in doc:
                if term not in self.index:
                    self.index[term] = [i]
                elif self.index[term][-1] != i:
                    self.index[term].append(i)
                    
    def get(self, q):
        return self.index.get(q, [])

    def delete(self, q):
        if q in self.index:
            del self.index[q]

In [31]:
import time

def timer(inv_index, field):
    st = time.time()
    inv = inv_index(field)
    print(f"Time taken for insertion : {time.time() - st}")
    st = time.time()
    for doc in df[field]:
        for term in doc:
            inv.get(term)
    print(f"Time taken for retrieval : {time.time() - st}")
    st = time.time()
    for doc in df[field]:
        for term in doc:
            inv.delete(term)
    print(f"Time taken for deletion : {time.time() - st}")

In [32]:
print("LIST")
timer(Inv_index_list, 'text_stem')

LIST
Time taken for insertion : 2.7705938816070557
Time taken for retrieval : 2.7776401042938232
Time taken for deletion : 1.2964794635772705


In [33]:
print("HASHMAP")
timer(Inv_index_dict, 'text_stem')

HASHMAP
Time taken for insertion : 0.012700319290161133
Time taken for retrieval : 0.009690284729003906
Time taken for deletion : 0.00861358642578125


In [34]:
print("BST")
timer(Inv_index_tree, 'text_stem')

BST
Time taken for insertion : 0.05147671699523926
Time taken for retrieval : 0.011879444122314453
Time taken for deletion : 0.0466768741607666


# Retrieval

In [35]:
def get_docs(index, query):
    print(query)
    if 'NOT' in query:
        docs = index.get(query.split(' ')[1])
        ans = []
        for i in range(100):
            if i not in docs:
                ans.append(i)
        return set(ans)
    return set(index.get(query))

def retrieve(index, query):
    docs = set([])
    op = ''
    for i,q in enumerate(query):
        if q in ['AND', 'OR']:
            op = q 
            continue
        if op == '':
            docs = get_docs(index, q)
        elif op == 'AND':
            docs = docs.intersection(get_docs(index, q))
        elif op == 'OR':
            docs = docs.union(get_docs(index, q))
    print(len(docs))
    return df.iloc[list(docs)]

def parse_query(query, process):
    ans = []
    nt = False
    for q in query.split(' '):
        if q in ['AND' ,'OR']:
            ans.append(q)
            continue
        if q == 'NOT':
            nt = True
            continue
        if nt:
            ans.append('NOT ' + process(q))
        else:
            ans.append(process(q))
        nt = False
    return ans

def search(index, query, process):
    return retrieve(index, parse_query(query, process))

def disp_results(docs):
    ans = []
    for ind in docs.index:
        display(docs['headline'][ind])
        display(docs['clean_text'][ind])
        print()

In [36]:
stem_index = Inv_index_dict('text_stem')
def search_dict(query):
    return search(stem_index, query, lambda x : stemming([x], porter_stemmer)[0])

## Query 1

In [37]:
results = search_dict('cricket AND stuart AND binny')
disp_results(results)

cricket
stuart
binni
1


'Stuart Binny announces retirement from first class and International cricket - The Indian Express'

'Stuart Binny India allrounder who owns the record of the best bowling figures in ODI format announced his retirement from cricket – firstclass and international – on Monday  Stuart 37 began his career with his state Karnataka and represented India in six Tests 14 ODIs and three Twenty20 Internationals  “The game of cricket runs through my blood and I will always look to give back to the game that has given me everything” said Stuart whose father Roger Binny was a member of the 1983 World Cup winning team “I just felt it’s time for me to move on from a player to coaching…and I have a lot to give back to cricket in that department… also now since I am a level 2 coach with the NCA he added  “I would like to inform you that I have decided to retire from firstclass and international cricket It has given me tremendous joy and pride to have represented my country at the highest international level”  Stuart still holds the record of the Best ODI Bowling by an Indian His figures of 44246 came 

In [38]:
results = search_dict('juice AND good AND health')
disp_results(results)

juic
good
health
1


'Want to improve your eyesight? Try these delicious and healthy dips - The Indian Express'

'Caring for your eyes is of paramount importance Most people have been working from home since the pandemic began and long working hours in front of computer screens have impacted their eyesight  Nutritionist Nmami Agarwal says certain nutrients can help to boost your eyesight She writes in her blog about some healthy dip recipes that will help to enhance your eye health Read on  “Eyefriendly nutrients such as lutein and zeaxanthin vitamin C vitamin E and zinc help in reducing the risk of certain eye diseases including agerelated macular degeneration and cataracts” she writes  ALSO READ  Ophthalmologist advises on how to use contact lenses correctly  View this post on Instagram A post shared by Nmami Life nmamilife   Chimichurri sauce  This recipe is rich in parsley and is high in vitamin A C and K which prove to be beneficial for eye health says the nutritionist Add apple cider vinegar olive oil and garlic and blend these ingredients in a blender Then add parsley cilantro salt and pep

In [39]:
results = search_dict('movie AND actor AND debut')
disp_results(results)

movi
actor
debut
2


'Reuters Entertainment News Summary - Devdiscourse'

'Following is a summary of current entertainment news briefs  Candyman Towers Over Box Office With Impressive 22 Million Debut  Candyman was summoned to the top of domestic box office charts collecting an impressive 2237 million from 3569 theaters in its first three days of release The Rrated slasher film written by Jordan Peele and directed by Captain Marvel 2 filmmaker Nia DaCosta surpassed industry expectations despite fears the delta variant would keep people from going to the movies Though multiplexes have struggled to sustain momentum as COVID19 cases continue to surge Hollywood has witnessed that new films available only in theaters such as Candyman and last weekends champ Free Guy have been seeing ticket sales hold steady compared to tentpoles being offered simultaneously on streaming services like The Suicide Squad and Hugh Jackmans Reminiscence   Actor Ed Asner star of Mary Tyler Moore Lou Grant dies at age 91  family  Ed Asner who played a gruff newsman for laughs and for dr

"Entertainment News Roundup: Actor Ed Asner, star of 'Mary Tyler Moore,' 'Lou Grant' dies at age 91; 'Candyman' Towers Over Box Office With Impressive $22 Million Debut and more - Devdiscourse"

'Following is a summary of current entertainment news briefs  Candyman Towers Over Box Office With Impressive 22 Million Debut  Candyman was summoned to the top of domestic box office charts collecting an impressive 2237 million from 3569 theaters in its first three days of release The Rrated slasher film written by Jordan Peele and directed by Captain Marvel 2 filmmaker Nia DaCosta surpassed industry expectations despite fears the delta variant would keep people from going to the movies Though multiplexes have struggled to sustain momentum as COVID19 cases continue to surge Hollywood has witnessed that new films available only in theaters such as Candyman and last weekends champ Free Guy have been seeing ticket sales hold steady compared to tentpoles being offered simultaneously on streaming services like The Suicide Squad and Hugh Jackmans Reminiscence   China cracks down on chaotic celebrity fan culture after scandals  China cracked down on what it described as a chaotic celebrity f

## Query 2

In [40]:
results = search_dict('MP OR politics AND NOT money')
disp_results(results)

MP
polit
NOT money
15


'Sports events mark birth anniversary of Dhyanchand - Times of India'

'Jhansi A number of sport events marked the 116th birth anniversary of legendary hockey player Padam Bhushan Major Dhyanchand in his home town Jhansi on Sunday the day which is also celebrated as National Sports DayIn addition a ‘Khel Prerak Samman’ award was declared by veteran Olympian and son of Dhyanchand Ashok Dhyanchand to be given from this year on this day for promoting sports A friendship match was held between Jhansi eleven and army eleven at Major Dhyanchand Astro Turf Hockey Stadium which was won by the former team The match was inaugurated by sports minister Upendra Tiwari in the presence of union state minister Bhanu Prasad Verma Jhansi Lalitpur MP Anurag Sharma and General Vipul Singhal GOC White Tiger Division Army while UP Chief Secretary Rajendra Kumar Tiwari was present virtually in the eventAll the invited veteran Olympians were felicitated on this occasion Army band gave also give grand performance by playing various tunes On this occasion Jhansi MP Anurag Sharma a

'Yati Narsinghanand Saraswati’s video where he calls women in politics as ‘rakhail’ of politicians goes viral, netizens demand action - OpIndia'

'On August 29 a video of Dasna priest Yati Narsinghanand Saraswati went viral on social media platforms in which he was heard using derogatory language against women in politics especially against those who are in BJP Several netizens strongly criticised him and demanded strict action  In the undated video Narsinghanand alleged that women in politics raise only after compromising with male leaders who are influential in the parties He said “The women you see in politics is or had been rakhail mistress of at least one male politician If she is not a mistress she is relative of an influential leader” He further said that during Samajwadi Party’s tenure the same situation continued However during Mayawati’s tenure it was not possible for male leaders to promote their mistresses he alleged  This is shocking I am really REALLY sorry I made an error of judgement in supporting this man pictwittercom9wwNOsnp6I — Shefali Vaidya 🇮🇳 ShefVaidya August 28 2021  He added “Then came the government of

...

'Ever since actor Sonu Sood met Delhi CM Arvind Kejriwal and was announced as the brand ambassador of the Desh Ke Mentor initiative speculations have been rife that he’s planning to get into this the field of politics However the actor wants to set the record straight that he is not joining any party  “I didn’t do this to get into politics I just wanted to help people” Sood tells us adding “When the whole movement of me helping people started in the first wave of the Covid pandemic last year even then these speculations started I want to send across a message that politics is a great profession and there’s no harm in case someone wants to desire to be in that”  The 48yearold adds that it’s not mandatory that one has to be a politician to help people  “Whatever I did in the past one and a half year it also conveys that even if you’re not into politics you can still help and motivate people That’s a message I want to tell people that don’t wait for an opportunity to help people just do i

'The Kerala Model That Is Really Broken Is Its Socio-Politics, Not Healthcare - Swarajya'

'The Kerala model has come under fire after the state started goofing up on Covid Despite a fall elsewhere in God’s Own Country Covid infection levels have refused to subside and the state continues to account for over 69 per cent of daily new cases and 56 per cent of all active cases in the country as on 30 August  This is however not enough to debunk the Kerala model which can loosely be said to be about high literacy and good quality healthcare services These continue to be the state’s strong points as is evident from the fatality rates in Covid With 20000andodd deaths from four million infections compared to 438000 deaths from 327 million cases allIndia Kerala’s fatality rate is less than 40 per cent of the national average of 13 per cent from official reported statistics This can hardly be called a failure of the Kerala educationcumhealthcare model  But the underlying assumptions behind this Kerala model — that this high performance on social indicators is led by high state spendi

'Narsinghanand makes derogatory comments against women in politics, video goes viral - The Siasat Daily'

'New Delhi Dasna priest Yati Narsinghanand Saraswati once against triggered controversy This time he allegedly made derogatory comments against women in politics  In the video which went viral on social media he can be seen calling women in politics ‘rakhail’ mistress He said “The women you see in politics is or had been rakhail of at least one male politician If she is not a mistress she is relative of an influential political leader”  He further alleged mistresses existed during Samajwadi Party rule However male politicians could not promote mistresses during Bahujan Samajwadi Party rule  In the later part of the video he can be seen making similar allegations on the women politicians in the ruling party too  This is shocking I am really REALLY sorry I made an error of judgement in supporting this man pictwittercom9wwNOsnp6I — Shefali Vaidya 🇮🇳 ShefVaidya August 28 2021  Reacting to the viral video netizens demand action against Narsinghanand  I request sharmarekha ji Uppolice to tak

In [41]:
results = search_dict('rs OR rupees AND NOT finance')
disp_results(results)

rs
rupe
NOT financ
8


'Sports events mark birth anniversary of Dhyanchand - Times of India'

'Jhansi A number of sport events marked the 116th birth anniversary of legendary hockey player Padam Bhushan Major Dhyanchand in his home town Jhansi on Sunday the day which is also celebrated as National Sports DayIn addition a ‘Khel Prerak Samman’ award was declared by veteran Olympian and son of Dhyanchand Ashok Dhyanchand to be given from this year on this day for promoting sports A friendship match was held between Jhansi eleven and army eleven at Major Dhyanchand Astro Turf Hockey Stadium which was won by the former team The match was inaugurated by sports minister Upendra Tiwari in the presence of union state minister Bhanu Prasad Verma Jhansi Lalitpur MP Anurag Sharma and General Vipul Singhal GOC White Tiger Division Army while UP Chief Secretary Rajendra Kumar Tiwari was present virtually in the eventAll the invited veteran Olympians were felicitated on this occasion Army band gave also give grand performance by playing various tunes On this occasion Jhansi MP Anurag Sharma a

'COVID-19 occupancies very less; prepared for any eventuality, says Narayana Health - CNBCTV18'

'Viren Shetty ED  Group COO Narayana Health said becuase of COVID their capex plans had been on hold but for the year they have modest plans of around Rs 250 crore with a potential to scale it up  Viren Shetty ED  Group COO at Narayana Health said at present their COVID occupancies were barely anything and vaccinations continued as normal In case of any eventuality he said the infrastructure has been set up  We bought all the ventilators we set apart all the COVID ICUs and should be in a position to take care of them Right now vaccinations continue as normal COVID occupancies are barely anything said Shetty  He added that COVID is a concern in the sense that it has not been fully eliminated from the country But I would echo what all the other hospital groups are saying that patients are not really coming to the hospital Those that come come very early and they do get treated Now we know a lot more about how to treat these patients than we did before he said  The overall COVID numbers h

...

'Reliance industries have placed an initial production order for the JioPhone Next 4G smartphone to UTL Neoluncs which is a joint venture between Bengalurubased UTL group and Neolync Solutions It has been approved to receive SOPs under the PLI scheme for handsets The company invested Rs 20 crore in Neolync solutions earlier this month Jio has set internal targets with distributors and financiers for sales  The Jio Exclusive smartphones are priced under Rs 10000 and have a nocost EMI of up to 6 months and additional benefits depending on the device Jio Exclusive smartphones will come with price support from the telco along with nocost EMI up to 6 months and additional benefits worth Rs 7000Rs 10000 depending upon the device ET Telecom in a recent report cited sources who noted that the Jio Exclusive bundled smartphones could extend their price range to mid and premium segment smartphones in the coming months  JioPhone Next Expected Specs  According to various leaks the smartphone will b

"India's sports poster girl Sindhu gets own special postal cover | Hyderabad News - Times of India - Times of India"

'HYDERABAD On the occasion of National Sports Day  a special postal cover in the name of ace badminton player and Tokyo Olympics bronze medallist P V Sindhu was released by the Telangana postal department at the shuttler’s home in the city on SundayLauding Sindhu for her dedication and achievements Telangana chief postmaster general S Rajendra Kumar said “This postal cover will motivate the youth It also commemorates the success of Indian sportspersons at the Tokyo Olympics”The special cover will be available for sale at Khairatabad Head Post Office and also online at httpswwwepostofficegovinbuyGueststampsaspxThe cost of Special Cover with cancellation is Rs 50 and without cancellation Rs 45 according to a press release from the postal department'

'KCR may renew efforts for third alternative in national politics - Business Standard'

'Telangana Chief Minister K Chandrasekhar Rao may revive his idea of a federal front and gear up to play a key role in as he is all set to lay the foundation stone for the office of Telangana Rashtra Samithi TRS in New Delhi next week  The development is not without significance and it comes at a time when Rao has just launched the Dalit Bandhu scheme which he hopes will become a torchbearer for empowerment of Dalits across the country  Looking to turn the event into a show of strength he will be leading a team of over 100 leaders to the capital All MPs of TRS state ministers state legislators and members of the party executive will attend the ceremony on September 2  The foundation stone laying ceremony has led to speculations in political circles that KCR as the TRS supremo is popularly known will formally announce his entry into the  KCRs ambition to play a key role in the national is wellknown He wants to cobble up a front with likeminded parties as an alternative to both the Bhara

In [42]:
results = search_dict('fever OR cold AND NOT covid')
disp_results(results)

fever
cold
NOT covid
5


'Entertainment calendar - Troy Record'

'Please email entertainment listings to entertainment518digitalfirstmediacom  This calendar is being regularly updated to reflect all event cancellations and postponements However it is recommended that readers contact event organizers before they attend For the most updated version of the calendar please visit saratogiancom or troyrecordcom  Thursday Aug 26 2021  EMPIRE LIVE 93 North Pearl St Albany Cold War Kids 7 pm empirelivealbanycom  CAFFE LENA 47 Phila St Saratoga Springs Peter Mulvey 7 pm caffelenaorg  BUTLER PARK 107 Gettle Rd Sand Lake Beatin’ The Odds 630 pm wwwtownofsandlakeus  SCHAGHTICOKE TOWN HALL 290 Northline Dr Melrose Refrigerators 630 pm townofschaghticokeorg  LIONS MUSIC PARK Iron Spring Park Front Street Ballston Spa Ballston Spa Community Jazz Band 6 pm ballstonorg  POSTPONED – THE EGG 1 Empire State Plaza Albany Nate Bargatze – Good Problem To Have 7 pm wwwtheeggorg  Friday Aug 27 2021  CAFFE LENA 47 Phila St Saratoga Springs Maeve Gilchrist 8 pm caffelenaorg  9

'Living space and health: Does urban design affect our health? - Medical News Today'

'Urban and housing design has intimate connections to health Poor design choices can worsen health with underlying issues of inequality a driving factor Share on Pinterest Where we live and the design of our living spaces have a significant impact on health Andrew ThomasGetty Images Article 25 of the Universal Declaration of Human Rights states “Everyone has the right to a standard of living adequate for the health and wellbeing of himself and of his family” This includes housing food clothing medical care social services and security if events beyond a person’s control affect their livelihood Researchers have shown that adequate housing has intimate links to a person’s physical and mental wellbeing According to the Executive Summary of the World Health Organization’s WHO Housing and Health Guidelines  poorly designed housing can increase the risk of trips falls injury isolation and stress for older people or people with disabilities Further insecure or unaffordable housing can exacerb

...

'Telangana Chief Minister K Chandrasekhar Rao may revive his idea of a federal front and gear up to play a key role in as he is all set to lay the foundation stone for the office of Telangana Rashtra Samithi TRS in New Delhi next week  The development is not without significance and it comes at a time when Rao has just launched the Dalit Bandhu scheme which he hopes will become a torchbearer for empowerment of Dalits across the country  Looking to turn the event into a show of strength he will be leading a team of over 100 leaders to the capital All MPs of TRS state ministers state legislators and members of the party executive will attend the ceremony on September 2  The foundation stone laying ceremony has led to speculations in political circles that KCR as the TRS supremo is popularly known will formally announce his entry into the  KCRs ambition to play a key role in the national is wellknown He wants to cobble up a front with likeminded parties as an alternative to both the Bhara

'UP should ensure health facilities to control fever in Mathura: Priyanka - Business Standard'

'After 29 cases of miteborne rickettsiosis known as scrub typhus were detected in Mathura General Secretary incharge Vadra on Monday demanded better health facilities in the state  She said in a tweet in Hindi Firozabad Mathura Agra and other places in UP children and adults have died due to fever which is heartwrenching UP government should step in immediately to control the disease and provide better health facilities  As per reports the health department has sounded an alert after the lab reports confirmed that 29 patients aged between two to 45 years had tested positive on Sunday  Scrub typhus is spread to people through bites of infected chiggers larval mites  The most common symptoms include fever headache body aches and sometimes rash In severe cases it can result in pneumonitis encephalitis mental changes ranging from confusion to coma congestive heart failure and circulatory collapse  Additional director of health department AK Singh said At least 29 cases of scrub typhus have

'ATEEZ main vocalist Jongho suffers knee injury, KQ Entertainment shares statement - Republic World'

'The vocal powerhouse of the South Korean boyband ATEEZ Jongho has suffered a minor knee injury Gearing up for their forthcoming release which will mark their seventh EP the youngest member of the idol band will reportedly sit out of major promotional events including a Japanese online event Dubbed as fourth Generation Leaders the group will perform with seven active members namely Hongjoong Leader Seonghwa Yunho Yeosang San Mingi and Wooyoung  ATEEZ Jonghos knee injury  Debuted under KQ Entertainment in 2018 the company released an official statement to relay the news of Jonghos minor knee injury to the Atiny fandom In the statement it was revealed that the 20yearold singer real name Choi Jongho was taken to the hospital after feeling uncomfortable in his knee Later he was diagnosed with a minor knee cartilage injury According to the statement the young singer will participate in limited events as he is suggested to get rest to fully recover Read the full statement released by KQ Ente